# Chain of Thought
We will use a variation of this technique by explicitly structuring our questions to guide the reasoning process. Before applying it, we will first verify that the API functions correctly.

In [1]:
import openai
from openai import OpenAI
import json
import re
import helper.creden as cre
client = OpenAI(api_key=cre.OPENAI_API_KEY)
from helper.helper import parse_conversation, select_random_test, get_completion_from_messages

# widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

client = OpenAI(api_key=cre.OPENAI_API_KEY)
model_selection="gpt-4o-mini"

In [2]:
messages_verification=[{"role": "user", "content": "Are you working?"}]
result = get_completion_from_messages(messages = messages_verification, model = model_selection, temperature=0.8, max_tokens_values=4)
result

"Yes, I'm here"

Now that we have confirmed the API is working, we will conduct an experiment.

In [3]:
tests_interviews = {
                "topics": [
                    {
                    "name": "Predictive Modeling",
                    "example": "Predicting customer churn for a subscription service.",
                    "problem": "Identifying which customers are likely to stop using the service and why."
                    },
                    {
                    "name": "Recommendation Systems",
                    "example": "Suggesting products on an e-commerce platform.",
                    "problem": "Providing personalized recommendations to improve user engagement and sales."
                    },
                    {
                    "name": "Natural Language Processing (NLP)",
                    "example": "Sentiment analysis on customer reviews.",
                    "problem": "Understanding customer sentiment to improve product offerings or service quality."
                    },
                    {
                    "name": "Time Series Analysis",
                    "example": "Forecasting monthly sales for inventory planning.",
                    "problem": "Predicting demand accurately to avoid stockouts or overstocking."
                    },
                    {
                    "name": "Anomaly Detection",
                    "example": "Detecting fraudulent transactions in banking systems.",
                    "problem": "Identifying unusual patterns or behaviors that indicate fraud."
                    },
                    {
                    "name": "Image and Video Analytics",
                    "example": "Detecting defects in manufacturing products using image classification.",
                    "problem": "Automating quality control to reduce manual inspection errors."
                    },
                    {
                    "name": "Customer Segmentation",
                    "example": "Grouping customers by purchasing behavior.",
                    "problem": "Creating targeted marketing campaigns to boost conversion rates."
                    },
                    {
                    "name": "Optimization Models",
                    "example": "Optimizing delivery routes for a logistics company.",
                    "problem": "Minimizing fuel costs and delivery time."
                    },
                    {
                    "name": "A/B Testing",
                    "example": "Testing two website layouts to improve user conversion.",
                    "problem": "Determining which design leads to better engagement or sales."
                    },
                    {
                    "name": "Big Data Processing",
                    "example": "Analyzing clickstream data from millions of users on a streaming platform.",
                    "problem": "Deriving insights and recommendations from massive, unstructured datasets."
                    }
                ]
                }
# select a random test
random_topic = select_random_test(tests_interviews)
random_topic

{'name': 'Predictive Modeling',
 'example': 'Predicting customer churn for a subscription service.',
 'problem': 'Identifying which customers are likely to stop using the service and why.'}

# Prompt generation

In [4]:
# Chain-of-Thought Prompting: 
prompt_example = f"""
We are conducting a data science interview focused exclusively on 
data science topics. Avoid discussing unrelated subjects. Ensure
a rigorous and structured interview process with the goal of 
evaluating the candidate's expertise in data science. Allocate
2 points for each well-answered question, and maintain high 
expectations as a data science professional with 5 years of
experience. Look for key terminology and concepts relevant
to the field.

Steps to Follow:
Introduction: Begin with a brief introduction of the company. Mention that it is a consultancy that collaborates with clients from various industries and fields.
Questioning: Ask one question at a time, ensuring it pertains to data science. Move on to the next question only after receiving an answer.
Example Problem: Incorporate this field example {random_topic["example"]} and this problem {random_topic["problem"]} as a case study to keep the discussion grounded.
Dataset Creation: Design a simple dataset with 5 rows and 4–8 features, including a target variable, to use as part of the problem-solving exercise. Present this dataset in a table format.
this its the example of the dataset
Question Design: Formulate one thoughtful question at a time based on the dataset or case study. If the question is unclear or unsuitable, reformulate it until it aligns with the goals of the interview.
Scoring: Assign a score from 0 to 9 for each response, where 9 represents expert-level knowledge. Evaluate based on detail, clarity, and the use of relevant data science concepts.
Conclusion: Conclude by thanking the candidate for their time and providing an overall score based on their performance.
Feedback: After completing all questions, identify areas where the interview process could be improved, providing specific points for enhancement.
Final Output: Summarize the results in a JSON format, including the overall score and interview feedback.

Reminder: Ensure the questions and evaluations are rigorous, leveraging your data science expertise to assess technical depth and problem-solving abilities.

"""

context = [{'role':'system', 'content':prompt_example}]
context

[{'role': 'system',
  'content': "\nWe are conducting a data science interview focused exclusively on \ndata science topics. Avoid discussing unrelated subjects. Ensure\na rigorous and structured interview process with the goal of \nevaluating the candidate's expertise in data science. Allocate\n2 points for each well-answered question, and maintain high \nexpectations as a data science professional with 5 years of\nexperience. Look for key terminology and concepts relevant\nto the field.\n\nSteps to Follow:\nIntroduction: Begin with a brief introduction of the company. Mention that it is a consultancy that collaborates with clients from various industries and fields.\nQuestioning: Ask one question at a time, ensuring it pertains to data science. Move on to the next question only after receiving an answer.\nExample Problem: Incorporate this field example Predicting customer churn for a subscription service. and this problem Identifying which customers are likely to stop using the servi

In [8]:
# Create text box for user input
user_input = widgets.Textarea(value='',
                              placeholder='Enter text here...',
                              description='User:',
                              rows=2,
                              layout=widgets.Layout(width='auto'),
                              disabled=False)

# Create text area for conversation display
conversation_history = widgets.Textarea(value='',
                                        description='Chat:',
                                     rows=15,
                                        layout=widgets.Layout(width='auto'))

# Create button for user to click to send their message
send_button = widgets.Button(description='Send')

# Initial Question (without response)
initial_question = "We will start with the interview. Can you give me a brief introduction about yourself?"
conversation_history.value += 'assistant: ' + initial_question + '\n'
context.append({'role': 'assistant', 'content': initial_question})

# -------------------------------
def send_button_clicked(b):
    """Send content to the chat"""
    global context
    
    conversation_history.value += 'user: ' + user_input.value + '\n'
    context.append({'role': 'user', 'content': user_input.value})
    response = get_completion_from_messages(messages = context, model = model_selection, temperature=0.1, max_tokens_values=2000)
    conversation_history.value += 'assistant: ' + response + '\n'
    context.append({'role': 'assistant', 'content': response})
    user_input.value = ''
    
# -------------------------------
send_button.on_click(send_button_clicked)

# -------------------------------
# Add keyboard shortcut for sending (Shift+Enter)
def handle_keypress(widget):
    if widget.value.endswith('\n'):
        send_button_clicked(None)
        
user_input.observe(handle_keypress, names='value')
# -------------------------------
# Display everything
display(conversation_history, user_input, send_button)

Textarea(value='assistant: We will start with the interview. Can you give me a brief introduction about yourse…

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='Send', style=ButtonStyle())

# Review of the interaction

In [9]:
json_result = parse_conversation(conversation_history.value)
print(json.dumps(json_result, indent=2))

{
  "conversation": [
    {
      "role": "assistant",
      "content": "We will start with the interview. Can you give me a brief introduction about yourself?"
    },
    {
      "role": "user",
      "content": "Im roishin"
    },
    {
      "role": "assistant",
      "content": "Thank you, Roishin. I appreciate your introduction.\nLet me provide a brief overview of our company. We are a consultancy that collaborates with clients from various industries and fields, leveraging data science to drive insights and improve decision-making.\nNow, let's dive into the interview.\n**Question 1:** Can you explain the concept of customer churn and why it is important for businesses to predict it?"
    },
    {
      "role": "user",
      "content": "refers to the percentage of cutomers(cx) who stop using a product or service over a given period"
    },
    {
      "role": "assistant",
      "content": "Thank you for your response, Roishin. You correctly defined customer churn as the percentage

In [11]:
pattern = r'\*\*Final Output:\*\*\s*```json\s*({.*?})\s*```'

match = re.search(pattern, conversation_history.value, re.DOTALL)

if match:
    extracted_data = match.group(1)
    print(f"Extracted Data:\n{extracted_data}")
else:
    print("No match found.")

Extracted Data:
{
  "overall_score": 5.8,
  "interview_feedback": {
    "strengths": [
      "Understanding of customer churn and its importance.",
      "Knowledge of evaluation metrics like F1 score and ROC AUC."
    ],
    "areas_for_improvement": [
      "Feature relevance in churn prediction.",
      "More detailed knowledge of data preprocessing techniques.",
      "Broader knowledge of machine learning algorithms."
    ]
  }
}


In this case, the model first reasons through the response and then assigns a score based on its evaluation. This approach ensures that the score is justified and directly linked to the accuracy and relevance of the answer.